In [1]:
# necessary imports
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import math
from time import time
from gdoctableapppy import gdoctableapp
import os
import openai
os.environ["OPENAI_API_KEY"] = openai_api_key
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from __future__ import print_function

import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from gdoctableapppy import gdoctableapp


d:\Users\fadia\anaconda3\envs\tf_gpu\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
t=time()
URL = " https://www.amazon.com/Arm-Hammer-Booster-Purifying-Waters/dp/B08T7X7XYX?pd_rd_w=UPYP1&content-id=amzn1.sym.80bfd2e5-c81d-48d5-ae85-c500e2214af9&pf_rd_p=80bfd2e5-c81d-48d5-ae85-c500e2214af9&pf_rd_r=P3BJ0XMXPK4K68KFG564&pd_rd_wg=a6oPV&pd_rd_r=9a304aff-d87b-4135-b186-63dca4d76e6c&pd_rd_i=B08T7X7XYX&psc=1&ref_=pd_bap_d_csi_pd_ys_c_rfy_rp_crs_6_t"
HEADERS = {
    "method": "GET",

    "accept-encoding": "gzip, deflate, br",

    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36",
    
}


response = requests.get(URL, headers=HEADERS)
#print(response.content)
# with open("myfile.html", "w") as file1:
#     # Writing data to a file
#     file1.write(str(response.content))

soup = BeautifulSoup(response.content, "html.parser")


see_all_reviews_link = soup.find("a", {"data-hook": "see-all-reviews-link-foot"})["href"]


URL_ALL_REVIEWS = "https://www.amazon.com/" + see_all_reviews_link


response = requests.get(URL_ALL_REVIEWS, headers=HEADERS)

soup = BeautifulSoup(response.content, "html.parser")


# Find the element that contains the ratings with reviews and extract the text
ratings_with_reviews_elem = soup.find("div", {"id": "filter-info-section"})
ratings_with_reviews_text = ratings_with_reviews_elem.get_text().strip()

#get just the reviews amount
reviews_count = re.search(r'(\d+,*\d*)\s+with reviews', ratings_with_reviews_text).group(1)

data = []


def scrape_reviews(page_url):
    """
    This function scrapes Amazon reviews from a given page URL.

    Args:
    - page_url: a string representing the URL of the page to be scraped.

    Returns:
    - page_data: a list of lists containing the scraped data. Each inner list contains 4 elements: the name of the reviewer,
    the title of the review, the body of the review, and the rating (number of stars) given by the reviewer.
    """

    # Send a GET request to the URL and retrieve the HTML content of the page
    response = requests.get(page_url, headers=HEADERS)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the review divs on the page
    reviews = soup.find_all('div', {'data-hook': 'review'})

    # Initialize an empty list to store the data scraped from this page
    page_data = []

    # Loop through each review div and extract the relevant information
    for review in reviews:
        # Extract the reviewer's name
        profile_name = review.find("span", {"class": "a-profile-name"}).get_text()

        # Extract the review title (if it exists)
        title_elem = review.find("a", {"data-hook": "review-title"})
        title = title_elem.get_text().strip() if title_elem else ""

        # Extract the review body
        body = review.find("span", {"data-hook": "review-body"}).get_text().strip()

        # Extract the rating (number of stars) given by the reviewer
        stars_elem = review.find("i", {"data-hook": "review-star-rating"})
        stars = float(stars_elem.span.get_text().split()[0]) if stars_elem and stars_elem.span else None
        if stars==None:
            stars_elem = review.find("i", {"data-hook": "cmps-review-star-rating"})
            stars = float(stars_elem.span.get_text().split()[0]) if stars_elem and stars_elem.span else None

        # Append the extracted information as a list to the page_data list
        page_data.append([profile_name, title, body, stars])

    # Return the page_data list containing the scraped information
    return page_data

# Create a ThreadPoolExecutor object to concurrently execute the scrape_reviews function on multiple pages
with ThreadPoolExecutor() as executor:
    # Calculate the number of pages to scrape based on the total number of reviews for the product
    page_num=math.ceil(float(reviews_count.replace(",",""))/10)

    # Generate a list of page URLs to scrape
    pages = [f"{URL_ALL_REVIEWS}+&pageNumber={i+1}" for i in range(page_num)]

    # Use the ThreadPoolExecutor to concurrently scrape the reviews from all the pages in the pages list
    results = executor.map(scrape_reviews, pages)

    # Combine the results from all the pages into a single list
    for result in results:
        data += result

In [3]:
df = pd.DataFrame(data, columns=['Name', 'Title', 'Body', 'Stars'])
df.head()

,Name,Title,Body,Stars
0,Parker S.,Cleaned my walls with this,I love this mixed in my mop bucket with warm w...,5.0
1,Karol Aguilar,good,"good alternative to other brands, but the scen...",5.0
2,ANIMALGIRL81,They smell so good.,Definitely make my laundry smell great! Will n...,5.0
3,Melissa Y Wood,Value,Smells great and decent price.,5.0
4,Foxylady,Terrific product to drive away bad smells,Great little laundry deodorant. Just enough to...,5.0


In [4]:
print(time()-t)

13.23202395439148


In [5]:
len(df)

806

In [6]:
# Instantiate OpenAI language model with desired temperature
llm = OpenAI(temperature=0)

# Instantiate a CharacterTextSplitter object to split text into chunks
text_splitter = CharacterTextSplitter()

# Define a function to generate summary for a given text
def generate_summary(all_text) -> str:
    """
    Generate summary for a given text by first splitting the text into smaller chunks, creating a Document object
    for each chunk, and then using a summarize chain to generate summary for each chunk. The function returns the
    combined summary for all the chunks.

    Parameters:
    all_text (str): The text for which summary is to be generated.

    Returns:
    str: The generated summary for the text.
    """
    # Split the text into chunks using the CharacterTextSplitter object
    texts = text_splitter.split_text(all_text)

    # Create a Document object for each chunk
    docs = [Document(page_content=t) for t in texts[:3]]

    # Load a summarize chain using the OpenAI language model
    chain = load_summarize_chain(llm, chain_type="map_reduce")

    # Generate summary for each chunk
    summary=chain.run(docs)

    return summary

# Define a function to generate summary for a list of text chunks
def chunks_summary(text_chunks):
    """
    Generate summary for a list of text chunks by first splitting the chunks into smaller chunks with a maximum length
    of 15,000 characters, generating summary for each smaller chunk, and then combining the summaries to generate a
    final summary for the entire list of text chunks.

    Parameters:
    text_chunks (list): A list of text chunks for which summary is to be generated.

    Returns:
    str: The generated summary for the entire list of text chunks.
    """
    # Split the chunks into smaller chunks with a maximum length of 15,000 characters
    text_chunks=chunk_data(text_chunks,15000)

    # Generate summary for each smaller chunk
    summaries = []
    for chunk in text_chunks:
        summary = generate_summary(chunk)
        summaries.append(summary)

    # Combine the summaries to generate the final summary for the entire list of text chunks
    if len(summaries)!=1:
        return chunks_summary(summaries)
    else:
        return summaries[0]

# Define a function to split data into chunks with a maximum length
def chunk_data(data, max_len) -> list: 
    """
    Split a list of data into chunks with a maximum length.

    Parameters:
    data (list): The list of data to be split into chunks.
    max_len (int): The maximum length of each chunk.

    Returns:
    list: The list of chunks.
    """
    res = []
    c_chuck = ""

    # Split each element of the list into smaller chunks with a maximum length of 'max_len'
    for review in data:
        if len(c_chuck+review)>max_len:
            if c_chuck:
                res.append(c_chuck)
                c_chuck = ""
        if len(review) > max_len:
            while len(review) > max_len:
                res.append(review[:max_len])
                review = review[max_len:]
            res.append(review)
            c_chuck = ""
        else:
            c_chuck+=review

    # Append any remaining text to the list of chunks
    if c_chuck:
        res.append(c_chuck)

    return res


# Generate summary for a list of text chunks
summary_of_product=chunks_summary(''.join(df['Body'].to_list()))

In [7]:
print(summary_of_product[1:])

Church & Hammer Clean Scentsations scent booster is an affordable and effective product that eliminates odors and provides a long-lasting, pleasant scent. Customers have been pleased with the product and will continue to purchase it.


In [8]:
df.head()

,Name,Title,Body,Stars
0,Parker S.,Cleaned my walls with this,I love this mixed in my mop bucket with warm w...,5.0
1,Karol Aguilar,good,"good alternative to other brands, but the scen...",5.0
2,ANIMALGIRL81,They smell so good.,Definitely make my laundry smell great! Will n...,5.0
3,Melissa Y Wood,Value,Smells great and decent price.,5.0
4,Foxylady,Terrific product to drive away bad smells,Great little laundry deodorant. Just enough to...,5.0


In [9]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

In [10]:
docs_service = build('drive', 'v3', credentials=creds)
file_metadata = {
    'name': 'My Document',
    'parents': ['1lBmUcDPDAiMqyHt3lBIZSvpOkPN5L8Hc'],
    'mimeType': 'application/vnd.google-apps.document'
}
document = docs_service.files().create(body=file_metadata).execute()
document_id=document['id']

In [11]:
# Build the Google Docs API client
docs_service = build('docs', 'v1', credentials=creds)


# Create the table
requests = [{
    'insertTable': {
        'rows': 1,
        'columns':4,
        'endOfSegmentLocation': {}
    }
}]

try:
    result = docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
except HttpError as e:
    print(e)

In [12]:
table_data=[]
table_data.append(df.columns.tolist())
# loop over the rows
for _, row in df.iterrows():
    table_data.append([row['Name'], row['Title'], row['Body'],row['Stars']])

table_data2 = [i[::-1] for i in table_data]

resource = {
    "oauth2": creds,
    "documentId": document_id,
    "tableIndex": 0,
    "values": table_data2
}

res = gdoctableapp.SetValues(resource)

In [13]:
drive_service = build('drive', 'v3', credentials=creds)

# Set up permission metadata
permission_metadata = {
    'role': 'reader',
    'type': 'anyone',
}

# Create the permission
permission = drive_service.permissions().create(
    fileId=document_id,
    body=permission_metadata,
    fields='id'
).execute()

# Get the permission ID
permission_id = permission['id']

# Generate the link
link = f'https://drive.google.com/file/d/{document_id}/view?usp=sharing'
print(link)

https://drive.google.com/file/d/1MhSm56enhZuyHdl0F-SSMoVON_bVGKy_6MGXrlyYpbU/view?usp=sharing
